In [ ]:
from dash import Dash, dcc, html, Input, Output
import plotly.express as px
import geopandas as gpd
import json
import numpy as np
from numpy import ma
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt

# FILEPATH MORTEN                                                                                                                           #RET FILEPATH HER!
filePath = "D:/data/CatchCrop/AllNoAlt/ccInfoNoAltProcessed.geojson"
# FILEPATH MADS
# filePath = "C:/Users/mkoli/Syddansk Universitet/Morten Thyrring Stouenberg - Speciale2023/Data/AllNoAlt/ccInfoNoAltProcessed.geojson"

with open(filePath, 'r', encoding='utf-8') as fp:
    ccData=json.load(fp)

gdf = gpd.GeoDataFrame.from_features(ccData)

# gdf['geometry'] = gpd.GeoSeries(gdf['geometry']).simplify(tolerance=0.001)

app = Dash(__name__)

styles = {
    'pre': {
        'border': 'thin lightgrey solid',
        'overflowX': 'scroll'
    }
}

app.layout = html.Div([
    html.Div(
        dcc.Graph(id="graph", style={'width': '130vh', 'height': '60vh'})
    ,style={'width': '74%', 'display': 'inline-block'}),
    html.Div([
        html.Table([
            html.Tr([html.Td(['Index of Selected Field']), html.Td(id='idxSelectedField')]),
            html.Tr([html.Td(['ID of Case for Selected Field']), html.Td(id='ktJNr')]),
            html.Tr([html.Td(['ID of Selected Field']), html.Td(id='ktMNr')]),
            html.Tr([html.Td(['Declared type of Catch Crop']), html.Td(id='fsAnmMrType')]),
            html.Tr([html.Td(['HA of Catch Crop Feild']), html.Td(id='fsAnmMarkHA')]),
            html.Tr([html.Td(['Date of Inspection']), html.Td(id='ktInspectionDate')]),
            html.Tr([html.Td(['Approved HA og Catch Crops']), html.Td(id='kt_ApprovedCC_HA')]),
            html.Tr([html.Td(['HA of Catch Crops Not Found']), html.Td(id='kt_NoCC_HA')]),
            html.Tr([html.Td(['HA of Catch Crops with Weak Cover']), html.Td(id='kt_WeakCoverCC_HA')]),
            html.Tr([html.Td(['HA of Catch Crops Early Destruction']), html.Td(id='kt_EarlyDestructionCC_HA')]),
            html.Tr([html.Td(['HA of Catch Crops Late Established']), html.Td(id='kt_LateEstablishedCC_HA')]),
            html.Tr([html.Td(['HA of Catch Crops with Wrong Species']), html.Td(id='kt_WrongSpeciesCC_HA')]),
        ]),
        dcc.Input(
            id = "idxInput",
            placeholder='Enter an index value...',
            type='number',
            value=0
        ),]
    ,style={'width': '24%', 'verticalAlign': 'top', 'display': 'inline-block', 'margin-top': '40'}),
    
    html.P('Time series (RGB)'),
    html.Div(
    dcc.Graph(id="imgPlot"),
    ),
    html.P('Time series (Cloud mask)'),
    html.Div(
    dcc.Graph(id="cloudPlotCLM"),
    ),
    html.P('Time series (cloud probability)'),
    html.Div(
    dcc.Graph(id="cloudPlotCLP"),
    ),
    html.P('Time series (NDVI))'),
    html.Div(
    dcc.Graph(id="ndviPlot"),
    ),
    html.P('Time series (NDVI MASKED))'),
    html.Div(
    dcc.Graph(id="ndvimasked"),
    ),
    html.P('Time series (NDVI MASKED by cloud Probability))'),
    html.Div(
    dcc.Graph(id="ndvimaskedP"),
    ),
])

@app.callback(
    Output('idxSelectedField', 'children'),
    Output('ktJNr', 'children'),
    Output('ktMNr', 'children'),
    Output('fsAnmMrType', 'children'),
    Output('fsAnmMarkHA', 'children'),
    Output('ktInspectionDate', 'children'),
    Output('kt_ApprovedCC_HA', 'children'),
    Output('kt_NoCC_HA', 'children'),
    Output('kt_WeakCoverCC_HA', 'children'),
    Output('kt_EarlyDestructionCC_HA', 'children'),
    Output('kt_LateEstablishedCC_HA', 'children'),
    Output('kt_WrongSpeciesCC_HA', 'children'),
    Input('graph', 'clickData'),
    Input('idxInput', 'value'))
def callback_a(clickData, idxInput):
    if clickData == None and idxInput == None :
        fieldIdx = 0

    elif clickData == None:
        fieldIdx = idxInput

    else:
        fieldIdx = clickData['points'][0]['pointIndex']

    fieldSelected = gdf.iloc[fieldIdx,]

    ktJNr = fieldSelected['ktJNr']
    ktMNr = fieldSelected['ktMNr']
    fsAnmMrType = fieldSelected['fsAnmMrType']
    fsAnmMarkHA = fieldSelected['fsAnmMarkHA']
    ktInspectionDate = fieldSelected['ktInspectionDate']
    kt_ApprovedCC_HA = fieldSelected['kt_ApprovedCC_HA']
    kt_NoCC_HA = fieldSelected['kt_NoCC_HA']
    kt_WeakCoverCC_HA = fieldSelected['kt_WeakCoverCC_HA']
    kt_EarlyDestructionCC_HA = fieldSelected['kt_EarlyDestructionCC_HA']
    kt_LateEstablishedCC_HA = fieldSelected['kt_LateEstablishedCC_HA']
    kt_WrongSpeciesCC_HA = fieldSelected['kt_WrongSpeciesCC_HA']

    return fieldIdx, ktJNr, ktMNr, fsAnmMrType, fsAnmMarkHA, ktInspectionDate, kt_ApprovedCC_HA, kt_NoCC_HA, kt_WeakCoverCC_HA, kt_EarlyDestructionCC_HA, kt_LateEstablishedCC_HA, kt_WrongSpeciesCC_HA

@app.callback(
    Output("graph", "figure"), 
    Input("idxInput", "value"))
def display_choropleth(idxSelectedField):
    fig = px.choropleth_mapbox(gdf,
                    geojson=gdf.geometry,
                    locations=gdf.index,
                    color="ccLabel",
                    center={"lat": 56.2, "lon": 11},
                    mapbox_style="open-street-map",
                    zoom=6.5)
    fig.update_geos(fitbounds="locations", visible=False)
    fig.update_layout(clickmode='event')
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    return fig


@app.callback(
    Output("imgPlot", "figure"), 
    Output("ndviPlot", "figure"),
    Output("cloudPlotCLM", "figure"),
    Output("cloudPlotCLP", "figure"),
    Output("ndvimasked", "figure"),
    Output("ndvimaskedP", "figure"),
    Input('graph', 'clickData'),
    Input('idxInput', 'value'))
def display_timeSeries(clickData, idxInput):
    if clickData == None and idxInput == None :
        fieldIdx = 0

    elif clickData == None:
        fieldIdx = idxInput

    else:
        fieldIdx = clickData['points'][0]['pointIndex']

    if fieldIdx < 500:
        fileN = 'ccImageIndex0_499.npy'
        fieldIdx = fieldIdx
    elif 500 <= fieldIdx < 1000:
        fileN = 'ccImageIndex500_999.npy'
        fieldIdx = fieldIdx - 500
    elif 1000 <= fieldIdx < 1500:
        fileN = 'ccImageIndex1000_1499.npy'
        fieldIdx = fieldIdx - 1000
    elif 1500 <= fieldIdx < 2000:
        fileN = 'ccImageIndex1500_1999.npy'
        fieldIdx = fieldIdx - 1500
    elif 2000 <= fieldIdx < 2500:
        fileN = 'ccImageIndex2000_2499.npy'
        fieldIdx = fieldIdx - 2000
    elif 2500 <= fieldIdx < 3000:
        fileN = 'ccImageIndex2500_2999.npy'
        fieldIdx = fieldIdx - 2500
    elif 3000 <= fieldIdx < 3500:
        fileN = 'ccImageIndex3000_3499.npy'
        fieldIdx = fieldIdx - 3000
    elif 3500 <= fieldIdx < 4000:
        fileN = 'ccImageIndex3500_3999.npy'
        fieldIdx = fieldIdx - 3500
    elif 4000 <= fieldIdx < 4500:
        fileN = 'ccImageIndex4000_4499.npy'
        fieldIdx = fieldIdx - 4000
    elif 4500 <= fieldIdx < 5000:
        fileN = 'ccImageIndex4500_4999.npy'
        fieldIdx = fieldIdx - 4500
    elif 5000 <= fieldIdx < 5500:
        fileN = 'ccImageIndex5000_5499.npy'
        fieldIdx = fieldIdx - 5000
    elif 5500 <= fieldIdx < 6000:
        fileN = 'ccImageIndex5500_5999.npy'
        fieldIdx = fieldIdx - 5500
    else:
        fileN = 'ccImageIndex6000_6234.npy'
        fieldIdx = fieldIdx - 6000

    # FILEPATH MORTEN                                                                                                                           #RET FILEPATH HER!
    filePath = f'D:/data/CatchCrop/AllNoAlt/{fileN}'
    # FILEPATH MADS                                                                                                                                                 
    # filePath = f'C:/Users/mkoli/Syddansk Universitet/Morten Thyrring Stouenberg - Speciale2023/Data/AllNoAlt/{fileN}'

 
    with open(filePath, 'rb') as fp:
        images=np.load(fp)

    # FILEPATH MORTEN                                                                                                                           #RET FILEPATH HER!
    filePath = "D:/data/CatchCrop/AllNoAlt/ccInfoNoAltProcessed.geojson"
    # FILEPATH MADS
    # filePath = "C:/Users/mkoli/Syddansk Universitet/Morten Thyrring Stouenberg - Speciale2023/Data/AllNoAlt/ccInfoNoAltProcessed.geojson"

    image = images[fieldIdx]

    with open(filePath, 'r', encoding='utf-8') as fp:
        ccData=json.load(fp)

    gdf = gpd.GeoDataFrame.from_features(ccData)
    
    titleSubPlots = []
    newLine = '<br>'

    for idx, item in enumerate(ccData['features'][fieldIdx]['properties']['imageDateTime']):
        period = str(ccData['features'][fieldIdx]['properties']['imagePeriod'][idx])
        split_text = f'acquisition period: {period[2:12]}{newLine} - {period[31:41]}{newLine}{newLine}Image time: {item[0:10]}{newLine}{item[11:19]}{newLine}{item[19:]}'
        titleSubPlots.append(split_text)

################################### RGB ###################################
    # RGB figure of time series of fields
    figRGB = make_subplots(
            rows=1, cols=9, 
            subplot_titles=titleSubPlots,
            vertical_spacing = 0.1
        )

    figRGB.update_annotations(font_size=10)

    for i in range(image.shape[0]):
        img_bgr = image[i, :, :, 0:3]
        img_rgb = np.flip(img_bgr, axis = 2)
        fig_img = px.imshow(3.5* img_rgb / 10000, zmin=0, zmax=1)
        figRGB.add_trace(fig_img.data[0], row=1, col=i+1)
        subPlotTitle = titleSubPlots[i]
        figRGB.layout.annotations[i].update(text=subPlotTitle)

################################### NDVI ###################################    
    # NDVI
    figNDVI = make_subplots(
        rows=1, cols=9,
        vertical_spacing = 0.1
    )

    figNDVI.update_annotations(font_size=10)

    for i in range(image.shape[0]):
        ndvi_image = (image[i, :, :, 3] - image[i, :, :, 2]) / (image[i, :, :, 3] + image[i, :, :, 2])
        fig_ndvi = px.imshow(ndvi_image, color_continuous_scale='RdBu_r')
        figNDVI.add_trace(fig_ndvi.data[0], row=1, col=i+1)

################################### Cloud Mask (CLM Band) ###################################
    # CloudMask (CLM) figure of time series of fields
    figCLM = make_subplots(
            rows=1, cols=9,
            vertical_spacing = 0.1
        )

    figCLM.update_annotations(font_size=10)

    for i in range(image.shape[0]):
         cloudMask = image[i, :, :, 8]
         cloudMask = np.where(cloudMask == 255, np.nan, cloudMask)
         fig_img = px.imshow(cloudMask)
         figCLM.add_trace(fig_img.data[0], row=1, col=i+1)

################################### Cloud Mask (CLP Band) ###################################
    # CloudMask (CLP) figure of time series of fields 
    figCLP = make_subplots(
            rows=1, cols=9,
            vertical_spacing = 0.1
        )

    figCLP.update_annotations(font_size=10)
    
    for i in range(image.shape[0]):
         cloudMask = image[i, :, :, 7] / 255
         cloudMask = np.where(cloudMask == 0, np.nan, cloudMask)
         fig_img = px.imshow(cloudMask)
         figCLP.add_trace(fig_img.data[0], row=1, col=i+1)


################################### Cloud Mask (CLM Band) ###################################
#Masked fields by cloudmask = 1
    figMASKED = make_subplots(
            rows=1, cols=9,
            vertical_spacing = 0.1
        )

    figMASKED.update_annotations(font_size=10)
        

    for i in range(image.shape[0]):
        ndvi_image = (image[i, :, :, 3] - image[i, :, :, 2]) / (image[i, :, :, 3] + image[i, :, :, 2])
        
        # create and apply mask for values equal to 1 or 255 in the 8th band
        mask = np.logical_or(image[i, :, :, 8] == 1, image[i, :, :, 8] == 255)
        masked_ndvi_image = np.ma.masked_array(ndvi_image, mask=mask)
        
        masked_ndvi_image = masked_ndvi_image.filled(np.nan)
        
        #Priiiiiiiiiint
        fig_ndvi = px.imshow(masked_ndvi_image, color_continuous_scale='RdBu_r')
        figMASKED.add_trace(fig_ndvi.data[0], row=1, col=i+1)


################################### Cloud Mask (CLP Band) ###################################
# Masked fields by cloud mask over specified tolerance 
    cloud_tolerance = 150
    
    figMASKEDP = make_subplots(
            rows=1, cols=9,
            vertical_spacing = 0.1,
            subplot_titles=(f'Cloud probability tolerance >= {(cloud_tolerance/255)*100}%','','','','','','','','')
        )

    figMASKEDP.update_annotations(font_size=10)
        

    for i in range(image.shape[0]):
        ndvi_image = (image[i, :, :, 3] - image[i, :, :, 2]) / (image[i, :, :, 3] + image[i, :, :, 2])
        
        # create and apply mask for cloud probability over a certain value
        mask = np.logical_and(image[i, :, :, 7] >= cloud_tolerance, image[i, :, :, 7] <= 256)
        masked_ndvi_image = np.ma.masked_array(ndvi_image, mask=mask)
        
        masked_ndvi_image = masked_ndvi_image.filled(np.nan)
        
        #Priiiiiiiiiint
        fig_ndvi = px.imshow(masked_ndvi_image, color_continuous_scale='RdBu_r')
        figMASKEDP.add_trace(fig_ndvi.data[0], row=1, col=i+1)

    return figRGB, figNDVI, figCLM, figCLP, figMASKED, figMASKEDP

################################### Run ###################################

if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)

